In [ ]:
import datasets
from src import data_loading

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-140k")
training_data = data_loading.load_training_data(dataset["train"].to_pandas())
print(f"Successfully loaded {len(training_data.entries)} entries")